In [3]:
import pyart
import datetime
import netCDF4

filename = '/net/denali/storage/radar/RELAMPAGO/DROPS/2018/12/14/cfrad.20181214_014605.932_to_20181214_014920.266_col-radar_REL_RHI45_RHI.nc'

radar = pyart.io.read(filename)
time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
time_text = time_start.isoformat()

The following test is for parallel computing

In [54]:
import sys
sys.path.append('/net/denali/storage2/radar2/tmp/Ivan/CSU/RELAMPAGO/src')

import radar_climatology as climatology
import os
import datetime
import netCDF4

import importlib
importlib.reload(climatology)

import glob
from pandas import DataFrame
import pyart

import multiprocessing as mp

def radar_statistics(file):
    radar = pyart.io.read(file)
    time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
    time_UTC = time_start.isoformat()
    try:
        echo_top = climatology.echo_top_drops(file)
        max_ref = climatology.max_reflectivity_drops(file)
        max_Kdp = climatology.max_Kdp_drops(file)
        max_Zdr = climatology.max_Zdr_drops(file)   
        min_Zdr = climatology.min_Zdr_drops(file)
    except:
        echo_top = -1
        max_ref = -100
        max_Kdp = -100
        max_Zdr = -100   
        min_Zdr = 100
    print(time_UTC)
    return (time_UTC, echo_top, max_ref, max_Kdp, max_Zdr, min_Zdr)
    
pool = mp.Pool(10)
path = '/net/denali/storage/radar/RELAMPAGO/DROPS/2018/11/10/'
files = [f for f in glob.glob(path + "**/*RHI*.nc", recursive=True)]
min_Zdr = []

results = pool.map_async(radar_statistics, files).get()
pool.close()
pool.join()

resutls = sorted(results, key = lambda x:x[0])

relampago_statistics = {'time_UTC':[],'echo_top':[], 
                        'max_ref': [], 'max_Kdp': [], 
                        'max_Zdr': [], 'min_Zdr':[]}

for i in range(0, len(results)):
    relampago_statistics['time_UTC'].append(results[i][0])
    relampago_statistics['echo_top'].append(results[i][1])
    relampago_statistics['max_ref'].append(results[i][2])
    relampago_statistics['max_Kdp'].append(results[i][3])
    relampago_statistics['max_Zdr'].append(results[i][4])
    relampago_statistics['min_Zdr'].append(results[i][5])

df = DataFrame(relampago_statistics, 
               columns= ['time_UTC', 'echo_top', 'max_ref',
                         'max_Kdp', 'max_Zdr', 'min_Zdr'])

export_excel = df.to_excel (r'./relampago_statistics_all_Drops.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path











2018-11-10T20:21:09
2018-11-10T20:28:31
2018-11-10T21:27:43
2018-11-10T20:23:08
2018-11-10T20:26:47
2018-11-10T21:33:45
2018-11-10T20:32:48
2018-11-10T22:00:59
2018-11-10T20:20:16
2018-11-10T21:08:04
2018-11-10T21:11:35
2018-11-10T21:08:59
2018-11-10T20:58:57
2018-11-10T20:56:20
2018-11-10T21:29:29
2018-11-10T22:26:15
2018-11-10T22:16:47
2018-11-10T21:35:30
2018-11-10T20:24:51
2018-11-10T21:46:55
2018-11-10T20:34:30
2018-11-10T21:05:25


KeyError: 'max_ref'

In [9]:
pool.close()

In [15]:
sorted_results = sorted(results, key = lambda x:x[0])

In [55]:
relampago_statistics = {'time_UTC':[],'echo_top':[], 
                        'max_ref': [], 'max_Kdp': [], 
                        'max_Zdr': [], 'min_Zdr':[]}

for i in range(0, len(results)):
    relampago_statistics['time_UTC'].append(results[i][0])
    relampago_statistics['echo_top'].append(results[i][1])
    relampago_statistics['max_ref'].append(results[i][2])
    relampago_statistics['max_Kdp'].append(results[i][3])
    relampago_statistics['max_Zdr'].append(results[i][4])
    relampago_statistics['min_Zdr'].append(results[i][5])

df = DataFrame(relampago_statistics, 
               columns= ['time_UTC', 'echo_top', 'max_ref',
                         'max_Kdp', 'max_Zdr', 'min_Zdr'])

export_excel = df.to_excel (r'./relampago_statistics_all_Drops.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path


In [53]:
len(results)

22